In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel('data v2.xlsx')

In [3]:
df = data.iloc[:, 1:]

df

,wk 1,wk 2,wk 3,wk 4,wk 5,wk 6,wk 7,wk 8,wk 9,wk 10,...,wk 53,wk 54,wk 55,wk 56,wk 57,wk 58,wk 59,wk 60,wk 61,wk 62
0,72848.0,57156.0,1099.0,71916.0,31700.0,65996.0,58028.0,60564.0,52239.0,24323.0,...,39966.0,41614.0,38862.0,14572.0,15601.0,8917.0,9358.0,24340.0,21677.0,19287.0
1,8067.0,9288.0,4099.0,8258.0,5709.0,10616.0,5009.0,6197.0,7866.0,9278.0,...,13640.0,11163.0,14303.0,8431.0,8099.0,4668.0,2701.0,4380.0,6482.0,8623.0
2,2914.0,7470.0,NaN,1202.0,12490.0,12119.0,14724.0,6245.0,1093.0,12820.0,...,5944.0,4720.0,4810.0,2805.0,2037.0,1754.0,2054.0,3074.0,4556.0,6626.0
3,138572.0,198611.0,96669.0,22963.0,104178.0,125459.0,132591.0,100540.0,161350.0,218670.0,...,116880.0,107120.0,93911.0,100258.0,1952778.0,1033274.0,1310199.0,1478926.0,2129291.0,1893435.0
4,9244.0,11096.0,9232.0,8208.0,7600.0,6944.0,4563.0,9776.0,8700.0,11534.0,...,4989.0,4932.0,5125.0,5017.0,33613.0,15980.0,20172.0,27206.0,33821.0,24743.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15785,11512.0,8245.0,12914.0,9970.0,12310.0,10472.0,10598.0,14704.0,20510.0,19010.0,...,4831.0,5320.0,5721.0,6560.0,21675.0,18270.0,30718.0,21174.0,30416.0,35850.0
15786,17760.0,17916.0,19246.0,29612.0,13052.0,12334.0,17690.0,11806.0,41632.0,40318.0,...,8172.0,8285.0,8355.0,11506.0,29772.0,23902.0,57171.0,17547.0,84238.0,110825.0
15787,4562.0,4597.0,5007.0,13070.0,13518.0,8152.0,8538.0,10700.0,15074.0,5568.0,...,4129.0,3632.0,3763.0,5170.0,13309.0,9453.0,20003.0,4305.0,2044.0,3551.0
15788,7652.0,13750.0,12949.0,14832.0,13820.0,10902.0,8427.0,16394.0,15428.0,14223.0,...,11800.0,8807.0,8998.0,9438.0,66732.0,43177.0,53816.0,53899.0,66497.0,54400.0


In [4]:
def has_nulls(dataset):
    print("Dataset has nulls is ", dataset.isnull().values.any())

has_nulls(df)

Dataset has nulls is  True


In [5]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)
imputer.fit(df)
df = pd.DataFrame(imputer.transform(df))

has_nulls(df)

Dataset has nulls is  False


In [6]:
def convert_data_anual(df):
    for i in range(0, 10):
        #df['wk '+ str(i)] = round((df['wk ' + str(i)]*2 + df['wk ' + str(int(i)+52)])/3, 2)
        #df[i] = round((df[i]*2 + df[i+52])/3, 2)
        df[i] = np.where(df[i] > df[i+52], df[i] , df[i+52])

    l = range(52, 62)
        
    #df.drop(columns = ['wk 53','wk 54','wk 55','wk 56','wk 57','wk 58','wk 59','wk 60','wk 61','wk 62'], inplace = True)
    df.drop(columns = l, inplace = True)
    return df

da = convert_data_anual(df)

In [7]:
da

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,72848.0,57156.0,38862.0,71916.0,31700.0,65996.0,58028.0,60564.0,52239.0,24323.0,...,65689.0,27310.0,28338.0,26669.0,43789.0,24282.0,53120.0,40401.0,31224.0,57982.0
1,13640.0,11163.0,14303.0,8431.0,8099.0,10616.0,5009.0,6197.0,7866.0,9278.0,...,22970.0,11753.0,18398.0,14626.0,16863.0,17231.0,15723.0,13696.0,12411.0,16156.0
2,5944.0,7470.0,4810.0,2805.0,12490.0,12119.0,14724.0,6245.0,4556.0,12820.0,...,4708.0,4674.0,4548.0,4386.0,4743.0,5379.0,6069.0,5402.0,6422.0,6444.0
3,138572.0,198611.0,96669.0,100258.0,1952778.0,1033274.0,1310199.0,1478926.0,2129291.0,1893435.0,...,1223526.0,207662.0,144686.0,126712.0,126905.0,131232.0,126687.0,119196.0,138916.0,109153.0
4,9244.0,11096.0,9232.0,8208.0,33613.0,15980.0,20172.0,27206.0,33821.0,24743.0,...,8776.0,5497.0,7344.0,6591.0,6244.0,6495.0,6428.0,4725.0,4626.0,6037.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15785,11512.0,8245.0,12914.0,9970.0,21675.0,18270.0,30718.0,21174.0,30416.0,35850.0,...,2628.0,1974.0,3433.0,5554.0,5500.0,5548.0,4730.0,3601.0,4565.0,5297.0
15786,17760.0,17916.0,19246.0,29612.0,29772.0,23902.0,57171.0,17547.0,84238.0,110825.0,...,9356.0,3544.0,7341.0,8585.0,8315.0,10648.0,8671.0,7774.0,7428.0,9050.0
15787,4562.0,4597.0,5007.0,13070.0,13518.0,9453.0,20003.0,10700.0,15074.0,5568.0,...,7478.0,4091.0,3794.0,4335.0,4476.0,4903.0,4075.0,3715.0,3825.0,4539.0
15788,11800.0,13750.0,12949.0,14832.0,66732.0,43177.0,53816.0,53899.0,66497.0,54400.0,...,9302.0,7960.0,11350.0,10880.0,10732.0,10248.0,9096.0,14410.0,12062.0,12887.0


In [43]:
dc = pd.DataFrame(da[0])

In [44]:
dc.head()

,0
0,72848.0
1,13640.0
2,5944.0
3,138572.0
4,9244.0


In [45]:
for i in range(1, 52):
    dc[str(i) + " - " + str(i+1)] = da[i]-da[i-1]

In [46]:
dc.drop(columns = [0], inplace = True)

In [47]:
dc.idxmax(axis = 1).unique()

array(['42 - 43', '10 - 11', '4 - 5', '22 - 23', '15 - 16', '26 - 27',
       '41 - 42', '6 - 7', '38 - 39', '36 - 37', '39 - 40', '17 - 18',
       '8 - 9', '35 - 36', '44 - 45', '31 - 32', '21 - 22', '19 - 20',
       '51 - 52', '7 - 8', '20 - 21', '1 - 2', '23 - 24', '43 - 44',
       '9 - 10', '29 - 30', '2 - 3', '16 - 17', '11 - 12', '12 - 13',
       '34 - 35', '5 - 6', '37 - 38', '28 - 29', '25 - 26', '30 - 31',
       '33 - 34', '32 - 33', '3 - 4', '27 - 28', '13 - 14', '18 - 19',
       '50 - 51', '24 - 25', '45 - 46', '40 - 41', '49 - 50', '47 - 48',
       '46 - 47', '48 - 49'], dtype=object)

In [48]:
dc.idxmax(axis = 1).value_counts()

4 - 5      9845
15 - 16    2896
41 - 42     687
8 - 9       200
39 - 40     181
6 - 7       166
7 - 8       149
9 - 10      121
1 - 2        93
19 - 20      89
33 - 34      81
38 - 39      76
17 - 18      74
16 - 17      68
5 - 6        59
22 - 23      57
32 - 33      55
35 - 36      54
31 - 32      52
37 - 38      50
34 - 35      49
3 - 4        46
30 - 31      45
10 - 11      43
21 - 22      40
11 - 12      40
36 - 37      39
42 - 43      39
28 - 29      36
2 - 3        36
29 - 30      35
43 - 44      34
20 - 21      32
44 - 45      30
13 - 14      28
23 - 24      28
24 - 25      22
12 - 13      17
26 - 27      14
25 - 26      14
27 - 28      11
51 - 52      11
18 - 19       9
45 - 46       8
50 - 51       6
46 - 47       6
48 - 49       6
40 - 41       5
49 - 50       5
47 - 48       3
Name: count, dtype: int64